#### DQN_closest_distance_path env

In [ ]:
# Register the environment so we can create it with gym.make()
gym.register(
    id=f"gymnasium_env/{env_name}",
    entry_point=ToyEnv_v2,
    max_episode_steps=300,  # Prevent infinite episodes. Here just set to 300 even though episode will terminate when stepping to last element of sequence
)
env = gym.make(f"gymnasium_env/{env_name}", coords_dict=coords_dict, max_visits=1)
# Create multiple environments for parallel training
# vec_env = gym.make_vec("gymnasium_env/SimpleTel-v0", num_envs=5, vectorization_mode='sync', Nf=Nf, target_sequence=true_sequence, nv_max=nv_max)

from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    check_env(env.unwrapped)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment has issues: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)


#### Sample framework

In [ ]:
"""
Universal Offline Reinforcement Learning Framework
Supports: DQN, DDPG, SAC, TD3, BC, CQL, IQL, and more
"""

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from abc import ABC, abstractmethod
from typing import Dict, Any, Tuple, Optional, List
from torch.utils.data import Dataset, DataLoader
from torch.distributions import Normal, Categorical


# ============================================================================
# 1. BASE DATASET (Same as before)
# ============================================================================

class BaseDataset(Dataset, ABC):
    """Abstract base class for offline RL datasets"""
    @abstractmethod
    def __len__(self) -> int:
        pass
    
    @abstractmethod
    def __getitem__(self, idx: int) -> Tuple:
        pass


class OfflineDataset(BaseDataset):
    """Universal offline dataset for any RL algorithm"""
    def __init__(self, states, actions, rewards, next_states, dones):
        self.states = np.array(states, dtype=np.float32)
        self.actions = np.array(actions, dtype=np.float32)
        self.rewards = np.array(rewards, dtype=np.float32)
        self.next_states = np.array(next_states, dtype=np.float32)
        self.dones = np.array(dones, dtype=np.float32)
        
    def __len__(self):
        return len(self.states)
    
    def __getitem__(self, idx):
        return (
            self.states[idx],
            self.actions[idx],
            self.rewards[idx],
            self.next_states[idx],
            self.dones[idx]
        )


# ============================================================================
# 2. NETWORK COMPONENTS
# ============================================================================

class MLP(nn.Module):
    """Multi-layer perceptron"""
    def __init__(self, input_dim, output_dim, hidden_dims=[256, 256], 
                 activation='relu', output_activation=None):
        super().__init__()
        
        activations = {
            'relu': nn.ReLU, 'tanh': nn.Tanh, 
            'elu': nn.ELU, 'leaky_relu': nn.LeakyReLU
        }
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(activations[activation]())
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, output_dim))
        
        if output_activation:
            layers.append(activations[output_activation]())
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)


class QNetwork(nn.Module):
    """Q-Network for DQN variants (discrete actions)"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256]):
        super().__init__()
        self.q_net = MLP(state_dim, action_dim, hidden_dims)
    
    def forward(self, states):
        return self.q_net(states)


class ContinuousQNetwork(nn.Module):
    """Q-Network for continuous actions (DDPG, TD3, SAC)"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256]):
        super().__init__()
        self.q_net = MLP(state_dim + action_dim, 1, hidden_dims)
    
    def forward(self, states, actions):
        x = torch.cat([states, actions], dim=-1)
        return self.q_net(x).squeeze(-1)


class DeterministicPolicy(nn.Module):
    """Deterministic policy for DDPG, TD3"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256], 
                 action_scale=1.0):
        super().__init__()
        self.policy = MLP(state_dim, action_dim, hidden_dims, output_activation='tanh')
        self.action_scale = action_scale
    
    def forward(self, states):
        return self.policy(states) * self.action_scale


class GaussianPolicy(nn.Module):
    """Gaussian policy for SAC, continuous BC"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256],
                 log_std_min=-20, log_std_max=2):
        super().__init__()
        self.log_std_min = log_std_min
        self.log_std_max = log_std_max
        
        self.backbone = MLP(state_dim, hidden_dims[-1], hidden_dims[:-1])
        self.mean_layer = nn.Linear(hidden_dims[-1], action_dim)
        self.log_std_layer = nn.Linear(hidden_dims[-1], action_dim)
    
    def forward(self, states):
        features = self.backbone(states)
        mean = self.mean_layer(features)
        log_std = self.log_std_layer(features)
        log_std = torch.clamp(log_std, self.log_std_min, self.log_std_max)
        return mean, log_std
    
    def sample(self, states):
        mean, log_std = self.forward(states)
        std = log_std.exp()
        normal = Normal(mean, std)
        x_t = normal.rsample()  # Reparameterization trick
        action = torch.tanh(x_t)
        
        # Compute log prob with tanh correction
        log_prob = normal.log_prob(x_t)
        log_prob -= torch.log(1 - action.pow(2) + 1e-6)
        log_prob = log_prob.sum(-1, keepdim=True)
        
        return action, log_prob


class CategoricalPolicy(nn.Module):
    """Categorical policy for discrete BC"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256]):
        super().__init__()
        self.policy = MLP(state_dim, action_dim, hidden_dims)
    
    def forward(self, states):
        logits = self.policy(states)
        return F.softmax(logits, dim=-1)
    
    def sample(self, states):
        probs = self.forward(states)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action, log_prob


# ============================================================================
# 3. BASE ALGORITHM INTERFACE
# ============================================================================

class BaseOfflineAlgorithm(ABC):
    """Abstract base class for all offline RL algorithms"""
    
    @abstractmethod
    def train_step(self, batch) -> Dict[str, float]:
        """Single training step, returns metrics"""
        pass
    
    @abstractmethod
    def select_action(self, state, deterministic=True):
        """Select action for evaluation"""
        pass
    
    @abstractmethod
    def save(self, filepath):
        pass
    
    @abstractmethod
    def load(self, filepath):
        pass


# ============================================================================
# 4. DQN FAMILY ALGORITHMS
# ============================================================================

class DQNAlgorithm(BaseOfflineAlgorithm):
    """Standard DQN / Double DQN for discrete actions"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256],
                 lr=3e-4, gamma=0.99, tau=0.005, double_q=True,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.action_dim = action_dim
        self.gamma = gamma
        self.tau = tau
        self.double_q = double_q
        self.device = device
        
        self.q_net = QNetwork(state_dim, action_dim, hidden_dims).to(device)
        self.target_q_net = QNetwork(state_dim, action_dim, hidden_dims).to(device)
        self.target_q_net.load_state_dict(self.q_net.state_dict())
        
        self.optimizer = optim.Adam(self.q_net.parameters(), lr=lr)
    
    def train_step(self, batch):
        states, actions, rewards, next_states, dones = [
            torch.FloatTensor(x).to(self.device) for x in batch
        ]
        actions = actions.long()
        
        # Current Q-values
        current_q = self.q_net(states).gather(1, actions.unsqueeze(1)).squeeze(1)
        
        # Target Q-values
        with torch.no_grad():
            if self.double_q:
                next_actions = self.q_net(next_states).argmax(1)
                next_q = self.target_q_net(next_states).gather(1, next_actions.unsqueeze(1)).squeeze(1)
            else:
                next_q = self.target_q_net(next_states).max(1)[0]
            
            target_q = rewards + (1 - dones) * self.gamma * next_q
        
        # Loss and optimization
        loss = F.mse_loss(current_q, target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        # Soft update target
        for param, target_param in zip(self.q_net.parameters(), self.target_q_net.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
        
        return {'loss': loss.item(), 'q_value': current_q.mean().item()}
    
    def select_action(self, state, deterministic=True):
        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            q_values = self.q_net(state)
            return q_values.argmax().item()
    
    def save(self, filepath):
        torch.save({'q_net': self.q_net.state_dict()}, filepath)
    
    def load(self, filepath):
        checkpoint = torch.load(filepath, map_location=self.device)
        self.q_net.load_state_dict(checkpoint['q_net'])


# ============================================================================
# 5. DDPG ALGORITHM (Continuous Control)
# ============================================================================

class DDPGAlgorithm(BaseOfflineAlgorithm):
    """Deep Deterministic Policy Gradient for continuous actions"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256],
                 actor_lr=3e-4, critic_lr=3e-4, gamma=0.99, tau=0.005,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.gamma = gamma
        self.tau = tau
        self.device = device
        
        # Actor
        self.actor = DeterministicPolicy(state_dim, action_dim, hidden_dims).to(device)
        self.target_actor = DeterministicPolicy(state_dim, action_dim, hidden_dims).to(device)
        self.target_actor.load_state_dict(self.actor.state_dict())
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=actor_lr)
        
        # Critic
        self.critic = ContinuousQNetwork(state_dim, action_dim, hidden_dims).to(device)
        self.target_critic = ContinuousQNetwork(state_dim, action_dim, hidden_dims).to(device)
        self.target_critic.load_state_dict(self.critic.state_dict())
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=critic_lr)
    
    def train_step(self, batch):
        states, actions, rewards, next_states, dones = [
            torch.FloatTensor(x).to(self.device) for x in batch
        ]
        
        # Update Critic
        with torch.no_grad():
            next_actions = self.target_actor(next_states)
            target_q = self.target_critic(next_states, next_actions)
            target_q = rewards + (1 - dones) * self.gamma * target_q
        
        current_q = self.critic(states, actions)
        critic_loss = F.mse_loss(current_q, target_q)
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        # Update Actor
        actor_loss = -self.critic(states, self.actor(states)).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Soft update targets
        self._soft_update(self.actor, self.target_actor)
        self._soft_update(self.critic, self.target_critic)
        
        return {
            'critic_loss': critic_loss.item(),
            'actor_loss': actor_loss.item(),
            'q_value': current_q.mean().item()
        }
    
    def _soft_update(self, source, target):
        for param, target_param in zip(source.parameters(), target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
    
    def select_action(self, state, deterministic=True):
        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            action = self.actor(state)
            return action.cpu().numpy()[0]
    
    def save(self, filepath):
        torch.save({
            'actor': self.actor.state_dict(),
            'critic': self.critic.state_dict()
        }, filepath)
    
    def load(self, filepath):
        checkpoint = torch.load(filepath, map_location=self.device)
        self.actor.load_state_dict(checkpoint['actor'])
        self.critic.load_state_dict(checkpoint['critic'])


# ============================================================================
# 6. SAC ALGORITHM (Maximum Entropy RL)
# ============================================================================

class SACAlgorithm(BaseOfflineAlgorithm):
    """Soft Actor-Critic for continuous actions"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256],
                 actor_lr=3e-4, critic_lr=3e-4, alpha=0.2, gamma=0.99, tau=0.005,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.gamma = gamma
        self.tau = tau
        self.alpha = alpha
        self.device = device
        
        # Actor (stochastic policy)
        self.actor = GaussianPolicy(state_dim, action_dim, hidden_dims).to(device)
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=actor_lr)
        
        # Twin Critics
        self.critic1 = ContinuousQNetwork(state_dim, action_dim, hidden_dims).to(device)
        self.critic2 = ContinuousQNetwork(state_dim, action_dim, hidden_dims).to(device)
        self.target_critic1 = ContinuousQNetwork(state_dim, action_dim, hidden_dims).to(device)
        self.target_critic2 = ContinuousQNetwork(state_dim, action_dim, hidden_dims).to(device)
        
        self.target_critic1.load_state_dict(self.critic1.state_dict())
        self.target_critic2.load_state_dict(self.critic2.state_dict())
        
        self.critic_optimizer = optim.Adam(
            list(self.critic1.parameters()) + list(self.critic2.parameters()), 
            lr=critic_lr
        )
    
    def train_step(self, batch):
        states, actions, rewards, next_states, dones = [
            torch.FloatTensor(x).to(self.device) for x in batch
        ]
        
        # Update Critics
        with torch.no_grad():
            next_actions, next_log_probs = self.actor.sample(next_states)
            target_q1 = self.target_critic1(next_states, next_actions)
            target_q2 = self.target_critic2(next_states, next_actions)
            target_q = torch.min(target_q1, target_q2) - self.alpha * next_log_probs.squeeze()
            target_q = rewards + (1 - dones) * self.gamma * target_q
        
        current_q1 = self.critic1(states, actions)
        current_q2 = self.critic2(states, actions)
        
        critic_loss = F.mse_loss(current_q1, target_q) + F.mse_loss(current_q2, target_q)
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        
        # Update Actor
        new_actions, log_probs = self.actor.sample(states)
        q1 = self.critic1(states, new_actions)
        q2 = self.critic2(states, new_actions)
        q = torch.min(q1, q2)
        
        actor_loss = (self.alpha * log_probs.squeeze() - q).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Soft update
        self._soft_update(self.critic1, self.target_critic1)
        self._soft_update(self.critic2, self.target_critic2)
        
        return {
            'critic_loss': critic_loss.item(),
            'actor_loss': actor_loss.item(),
            'q_value': current_q1.mean().item()
        }
    
    def _soft_update(self, source, target):
        for param, target_param in zip(source.parameters(), target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)
    
    def select_action(self, state, deterministic=True):
        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            if deterministic:
                mean, _ = self.actor(state)
                action = torch.tanh(mean)
            else:
                action, _ = self.actor.sample(state)
            return action.cpu().numpy()[0]
    
    def save(self, filepath):
        torch.save({'actor': self.actor.state_dict()}, filepath)
    
    def load(self, filepath):
        checkpoint = torch.load(filepath, map_location=self.device)
        self.actor.load_state_dict(checkpoint['actor'])


# ============================================================================
# 7. BEHAVIORAL CLONING (Imitation Learning)
# ============================================================================

class BehavioralCloningAlgorithm(BaseOfflineAlgorithm):
    """Behavioral Cloning - simple supervised learning"""
    def __init__(self, state_dim, action_dim, hidden_dims=[256, 256],
                 lr=3e-4, discrete=True,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.discrete = discrete
        self.device = device
        
        if discrete:
            self.policy = CategoricalPolicy(state_dim, action_dim, hidden_dims).to(device)
        else:
            self.policy = GaussianPolicy(state_dim, action_dim, hidden_dims).to(device)
        
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
    
    def train_step(self, batch):
        states, actions, _, _, _ = [torch.FloatTensor(x).to(self.device) for x in batch]
        
        if self.discrete:
            actions = actions.long()
            probs = self.policy(states)
            loss = F.cross_entropy(probs, actions)
        else:
            mean, log_std = self.policy(states)
            std = log_std.exp()
            loss = F.mse_loss(mean, actions)  # Simple MSE for continuous
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return {'loss': loss.item()}
    
    def select_action(self, state, deterministic=True):
        with torch.no_grad():
            state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
            if self.discrete:
                probs = self.policy(state)
                return probs.argmax().item()
            else:
                mean, _ = self.policy(state)
                return mean.cpu().numpy()[0]
    
    def save(self, filepath):
        torch.save({'policy': self.policy.state_dict()}, filepath)
    
    def load(self, filepath):
        checkpoint = torch.load(filepath, map_location=self.device)
        self.policy.load_state_dict(checkpoint['policy'])


# ============================================================================
# 8. AGENT WRAPPER (High-Level Interface)
# ============================================================================

class Agent:
    """
    High-level agent wrapper that provides a convenient interface for:
    - Training
    - Evaluation  
    - Saving/Loading
    - Action selection with exploration
    - Logging and monitoring
    """
    def __init__(
        self,
        algorithm: BaseOfflineAlgorithm,
        env=None,
        name: str = "agent"
    ):
        self.algorithm = algorithm
        self.env = env
        self.name = name
        self.training_history = []
        self.eval_history = []
    
    def train(
        self,
        dataset: BaseDataset,
        num_epochs: int,
        batch_size: int = 256,
        log_interval: int = 10,
        eval_interval: Optional[int] = None,
        eval_episodes: int = 5,
        save_best: bool = True,
        save_dir: str = "./checkpoints"
    ):
        """
        Train the agent with optional periodic evaluation
        
        Args:
            dataset: Offline dataset
            num_epochs: Number of training epochs
            batch_size: Batch size
            log_interval: Log every N epochs
            eval_interval: Evaluate every N epochs (None = no eval)
            eval_episodes: Number of episodes for evaluation
            save_best: Save best model during training
            save_dir: Directory to save checkpoints
        """
        import os
        os.makedirs(save_dir, exist_ok=True)
        
        trainer = UniversalTrainer(self.algorithm, dataset, batch_size)
        
        best_reward = float('-inf')
        
        print(f"Training {self.name} for {num_epochs} epochs...")
        print("=" * 70)
        
        for epoch in range(num_epochs):
            # Train one epoch
            epoch_metrics = {}
            for batch in trainer.dataloader:
                metrics = self.algorithm.train_step(batch)
                for key, value in metrics.items():
                    if key not in epoch_metrics:
                        epoch_metrics[key] = []
                    epoch_metrics[key].append(value)
            
            # Average metrics
            avg_metrics = {k: np.mean(v) for k, v in epoch_metrics.items()}
            self.training_history.append({'epoch': epoch, 'metrics': avg_metrics})
            
            # Logging
            if (epoch + 1) % log_interval == 0:
                metric_str = ', '.join([f"{k}: {v:.4f}" for k, v in avg_metrics.items()])
                print(f"Epoch {epoch + 1}/{num_epochs} | {metric_str}")
            
            # Periodic evaluation
            if eval_interval and (epoch + 1) % eval_interval == 0 and self.env:
                print(f"\n--- Evaluation at epoch {epoch + 1} ---")
                avg_reward, std_reward = self.evaluate(
                    num_episodes=eval_episodes,
                    verbose=False
                )
                self.eval_history.append({
                    'epoch': epoch,
                    'avg_reward': avg_reward,
                    'std_reward': std_reward
                })
                print(f"Reward: {avg_reward:.2f} ± {std_reward:.2f}\n")
                
                # Save best model
                if save_best and avg_reward > best_reward:
                    best_reward = avg_reward
                    save_path = os.path.join(save_dir, f"{self.name}_best.pt")
                    self.save(save_path)
                    print(f"💾 Saved best model (reward: {best_reward:.2f})")
        
        print("=" * 70)
        print(f"Training complete!")
        
        # Final save
        final_path = os.path.join(save_dir, f"{self.name}_final.pt")
        self.save(final_path)
        print(f"💾 Saved final model to {final_path}")
        
        return self.training_history
    
    def evaluate(
        self,
        env=None,
        num_episodes: int = 10,
        deterministic: bool = True,
        render: bool = False,
        verbose: bool = True
    ):
        """
        Evaluate the agent
        
        Args:
            env: Environment (uses self.env if None)
            num_episodes: Number of episodes
            deterministic: Use deterministic policy
            render: Render environment
            verbose: Print results
        
        Returns:
            (avg_reward, std_reward)
        """
        env = env or self.env
        if env is None:
            raise ValueError("No environment provided for evaluation")
        
        evaluator = UniversalEvaluator(self.algorithm, env)
        
        if verbose:
            return evaluator.evaluate(num_episodes, deterministic)
        else:
            # Silent evaluation
            total_rewards = []
            for _ in range(num_episodes):
                state, _ = env.reset()
                episode_reward = 0
                done = False
                
                while not done:
                    if render:
                        env.render()
                    action = self.algorithm.select_action(state, deterministic)
                    next_state, reward, terminated, truncated, _ = env.step(action)
                    done = terminated or truncated
                    episode_reward += reward
                    state = next_state
                
                total_rewards.append(episode_reward)
            
            return np.mean(total_rewards), np.std(total_rewards)
    
    def act(self, state, deterministic: bool = True, epsilon: float = 0.0):
        """
        Select action with optional epsilon-greedy exploration
        
        Args:
            state: Current state
            deterministic: Use deterministic policy
            epsilon: Exploration probability (for epsilon-greedy)
        
        Returns:
            action
        """
        # Epsilon-greedy exploration (mainly for DQN)
        if epsilon > 0 and np.random.random() < epsilon:
            # Random action
            if hasattr(self.env.action_space, 'n'):
                # Discrete
                return np.random.randint(0, self.env.action_space.n)
            else:
                # Continuous
                return self.env.action_space.sample()
        
        return self.algorithm.select_action(state, deterministic)
    
    def save(self, filepath: str):
        """Save agent to file"""
        self.algorithm.save(filepath)
        print(f"Agent saved to {filepath}")
    
    def load(self, filepath: str):
        """Load agent from file"""
        self.algorithm.load(filepath)
        print(f"Agent loaded from {filepath}")
    
    def get_training_history(self):
        """Get training history as dict"""
        return self.training_history
    
    def get_eval_history(self):
        """Get evaluation history as dict"""
        return self.eval_history
    
    def plot_training(self):
        """Plot training curves (requires matplotlib)"""
        try:
            import matplotlib.pyplot as plt
            
            if not self.training_history:
                print("No training history to plot")
                return
            
            # Extract metrics
            epochs = [h['epoch'] for h in self.training_history]
            metrics = self.training_history[0]['metrics'].keys()
            
            fig, axes = plt.subplots(1, len(metrics), figsize=(5*len(metrics), 4))
            if len(metrics) == 1:
                axes = [axes]
            
            for ax, metric in zip(axes, metrics):
                values = [h['metrics'][metric] for h in self.training_history]
                ax.plot(epochs, values)
                ax.set_xlabel('Epoch')
                ax.set_ylabel(metric.replace('_', ' ').title())
                ax.set_title(f'{metric.replace("_", " ").title()} over Training')
                ax.grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig(f"{self.name}_training.png", dpi=150, bbox_inches='tight')
            print(f"Training plot saved to {self.name}_training.png")
            plt.show()
            
        except ImportError:
            print("matplotlib not installed. Cannot plot training curves.")
    
    def plot_evaluation(self):
        """Plot evaluation curves (requires matplotlib)"""
        try:
            import matplotlib.pyplot as plt
            
            if not self.eval_history:
                print("No evaluation history to plot")
                return
            
            epochs = [h['epoch'] for h in self.eval_history]
            rewards = [h['avg_reward'] for h in self.eval_history]
            stds = [h['std_reward'] for h in self.eval_history]
            
            plt.figure(figsize=(8, 5))
            plt.plot(epochs, rewards, marker='o', label='Average Reward')
            plt.fill_between(epochs, 
                           np.array(rewards) - np.array(stds),
                           np.array(rewards) + np.array(stds),
                           alpha=0.3, label='±1 Std Dev')
            plt.xlabel('Epoch')
            plt.ylabel('Reward')
            plt.title('Evaluation Performance')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f"{self.name}_evaluation.png", dpi=150, bbox_inches='tight')
            print(f"Evaluation plot saved to {self.name}_evaluation.png")
            plt.show()
            
        except ImportError:
            print("matplotlib not installed. Cannot plot evaluation curves.")
    
    def __repr__(self):
        return f"Agent(name={self.name}, algorithm={self.algorithm.__class__.__name__})"


# ============================================================================
# 9. UNIVERSAL TRAINER
# ============================================================================

class UniversalTrainer:
    """Universal trainer for any offline RL algorithm"""
    def __init__(self, algorithm: BaseOfflineAlgorithm, dataset: BaseDataset,
                 batch_size: int = 256, num_workers: int = 0):
        self.algorithm = algorithm
        self.dataloader = DataLoader(
            dataset, batch_size=batch_size, 
            shuffle=True, num_workers=num_workers
        )
    
    def train(self, num_epochs: int, log_interval: int = 10):
        """Train algorithm on offline dataset"""
        print(f"Training for {num_epochs} epochs...")
        
        history = {'epoch': [], 'metrics': []}
        
        for epoch in range(num_epochs):
            epoch_metrics = {}
            
            for batch in self.dataloader:
                metrics = self.algorithm.train_step(batch)
                
                for key, value in metrics.items():
                    if key not in epoch_metrics:
                        epoch_metrics[key] = []
                    epoch_metrics[key].append(value)
            
            # Average metrics
            avg_metrics = {k: np.mean(v) for k, v in epoch_metrics.items()}
            history['epoch'].append(epoch)
            history['metrics'].append(avg_metrics)
            
            if (epoch + 1) % log_interval == 0:
                metric_str = ', '.join([f"{k}: {v:.4f}" for k, v in avg_metrics.items()])
                print(f"Epoch {epoch + 1}/{num_epochs} | {metric_str}")
        
        return history


# ============================================================================
# 9. UNIVERSAL EVALUATOR
# ============================================================================

class UniversalEvaluator:
    """Universal evaluator for any RL algorithm"""
    def __init__(self, algorithm: BaseOfflineAlgorithm, env):
        self.algorithm = algorithm
        self.env = env
    
    def evaluate(self, num_episodes: int = 10, deterministic: bool = True):
        """Evaluate algorithm"""
        total_rewards = []
        
        for episode in range(num_episodes):
            state, _ = self.env.reset()
            episode_reward = 0
            done = False
            
            while not done:
                action = self.algorithm.select_action(state, deterministic)
                next_state, reward, terminated, truncated, _ = self.env.step(action)
                done = terminated or truncated
                
                episode_reward += reward
                state = next_state
            
            total_rewards.append(episode_reward)
        
        avg_reward = np.mean(total_rewards)
        std_reward = np.std(total_rewards)
        
        print(f"\nEvaluation ({num_episodes} episodes):")
        print(f"Average Reward: {avg_reward:.2f} ± {std_reward:.2f}")
        
        return avg_reward, std_reward


# ============================================================================
# 11. USAGE EXAMPLES WITH AGENT WRAPPER
# ============================================================================

if __name__ == "__main__":
    import gymnasium as gym
    
    print("=" * 70)
    print("UNIVERSAL OFFLINE RL FRAMEWORK WITH AGENT WRAPPER")
    print("=" * 70)
    
    # ========================================================================
    # EXAMPLE 1: DQN Agent - Simple Usage
    # ========================================================================
    print("\n[EXAMPLE 1] DQN Agent - Simple Training")
    print("-" * 70)
    
    env = gym.make('CartPole-v1')
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    
    # Collect data
    print("Collecting offline data...")
    states, actions, rewards, next_states, dones = [], [], [], [], []
    for _ in range(50):
        state, _ = env.reset()
        done = False
        while not done:
            action = env.action_space.sample()
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            next_states.append(next_state)
            dones.append(float(done))
            state = next_state
    
    dataset = OfflineDataset(states, actions, rewards, next_states, dones)
    print(f"Collected {len(dataset)} transitions")
    
    # Create agent with DQN
    dqn_algorithm = DQNAlgorithm(state_dim, action_dim, double_q=True)
    agent = Agent(dqn_algorithm, env=env, name="dqn_cartpole")
    
    # Train with automatic evaluation and saving
    agent.train(
        dataset=dataset,
        num_epochs=30,
        batch_size=64,
        log_interval=5,
        eval_interval=10,  # Evaluate every 10 epochs
        eval_episodes=5,
        save_best=True
    )
    
    # Final evaluation
    print("\n" + "=" * 70)
    print("FINAL EVALUATION")
    print("=" * 70)
    agent.evaluate(num_episodes=20)
    
    env.close()
    
    # ========================================================================
    # EXAMPLE 2: DDPG Agent - Advanced Usage
    # ========================================================================
    print("\n" + "=" * 70)
    print("[EXAMPLE 2] DDPG Agent - Advanced Usage")
    print("-" * 70)
    
    env = gym.make('Pendulum-v1')
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    
    # Collect data
    print("Collecting offline data...")
    states, actions, rewards, next_states, dones = [], [], [], [], []
    for _ in range(50):
        state, _ = env.reset()
        done = False
        while not done:
            action = env.action_space.sample()
            next_state, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            next_states.append(next_state)
            dones.append(float(done))
            state = next_state
    
    dataset = OfflineDataset(states, actions, rewards, next_states, dones)
    print(f"Collected {len(dataset)} transitions")
    
    # Create DDPG agent
    ddpg_algorithm = DDPGAlgorithm(state_dim, action_dim, actor_lr=1e-3, critic_lr=1e-3)
    agent = Agent(ddpg_algorithm, env=env, name="ddpg_pendulum")
    
    # Train
    agent.train(
        dataset=dataset,
        num_epochs=30,
        batch_size=128,
        log_interval=5,
        eval_interval=10,
        save_best=True
    )
    
    # Evaluate
    agent.evaluate(num_episodes=10)
    
    env.close()
    
    # ========================================================================
    # EXAMPLE 3: Agent Usage Patterns
    # ========================================================================
    print("\n" + "=" * 70)
    print("[EXAMPLE 3] Agent Usage Patterns")
    print("-" * 70)
    
    env = gym.make('CartPole-v1')
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.n
    
    # Create and train agent
    bc_algorithm = BehavioralCloningAlgorithm(state_dim, action_dim, discrete=True)
    agent = Agent(bc_algorithm, env=env, name="bc_cartpole")
    
    # Reuse dataset
    dataset = OfflineDataset(states[:500], actions[:500], rewards[:500], 
                            next_states[:500], dones[:500])
    
    print("\n1. Training without evaluation:")
    agent.train(dataset, num_epochs=20, batch_size=64, log_interval=5)
    
    print("\n2. Using agent for inference:")
    state, _ = env.reset()
    for step in range(5):
        action = agent.act(state, deterministic=True)
        print(f"Step {step}: state={state[:2]}, action={action}")
        state, _, terminated, truncated, _ = env.step(action)
        if terminated or truncated:
            break
    
    print("\n3. Save and load:")
    agent.save("my_agent.pt")
    
    # Create new agent and load weights
    new_agent = Agent(
        BehavioralCloningAlgorithm(state_dim, action_dim, discrete=True),
        env=env,
        name="loaded_agent"
    )
    new_agent.load("my_agent.pt")
    
    print("\n4. Evaluate loaded agent:")
    new_agent.evaluate(num_episodes=5)
    
    print("\n5. Access training history:")
    history = agent.get_training_history()
    print(f"Training ran for {len(history)} epochs")
    if history:
        print(f"Final metrics: {history[-1]['metrics']}")
    
    env.close()
    
    # ========================================================================
    # EXAMPLE 4: Comparing Multiple Agents
    # ========================================================================
    print("\n" + "=" * 70)
    print("[EXAMPLE 4] Comparing Multiple Agents")
    print("-" * 70)
    
    env = gym.make('CartPole-v1')
    
    algorithms = {
        'DQN': DQNAlgorithm(state_dim, action_dim, double_q=False),
        'DoubleDQN': DQNAlgorithm(state_dim, action_dim, double_q=True),
        'BC': BehavioralCloningAlgorithm(state_dim, action_dim, discrete=True)
    }
    
    results = {}
    
    for name, algorithm in algorithms.items():
        print(f"\nTraining {name}...")
        agent = Agent(algorithm, env=env, name=name.lower())
        agent.train(dataset, num_epochs=15, batch_size=64, log_interval=5)
        avg_reward, std_reward = agent.evaluate(num_episodes=10, verbose=False)
        results[name] = {'avg': avg_reward, 'std': std_reward}
        print(f"{name} - Reward: {avg_reward:.2f} ± {std_reward:.2f}")
    
    print("\n" + "=" * 70)
    print("COMPARISON RESULTS")
    print("-" * 70)
    for name, result in results.items():
        print(f"{name:15s}: {result['avg']:6.2f} ± {result['std']:.2f}")
    
    best_agent = max(results.items(), key=lambda x: x[1]['avg'])[0]
    print(f"\n🏆 Best algorithm: {best_agent}")
    
    env.close()
    
    print("\n" + "=" * 70)
    print("ALL EXAMPLES COMPLETE")
    print("=" * 70)

vectorized env

In [ ]:
# class MultiTaskSimpleTelEnv(SimpleTelEnv):
#     def __init__(self):
#         super().__init__()

#     def set_mu(self, new_mu: float):
#         self.mu = new_mu

# env = gym.make("gymnasium_env/SimpleTel-v0", Nf=Nf, target_sequence=true_sequence, nv_max=nv_max)
# vec_env = make_vec_env(MultiTaskSimpleTelEnv)
# vec_env.env_method.has_attr("mu")
# vec_env.env_method("set_wrapper_attr", "mu", .1)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random

import gymnasium as gym
from gymnasium.spaces import Dict, Box, Discrete

from stable_baselines3 import DQN, PPO, A2C
# from sb3_contrib import TRPO, CrossQ, ARS, QRDQN, TQC
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

import optuna

import os
import time
import pickle
import sys
sys.path.append('../src/')
from environments import SimpleTelEnv
from utils import save_results, load_results

In [6]:
env_name = 'SimpleTel-v0'

OUTDIR = f'results/{env_name}/'
if not os.path.exists(OUTDIR):
    os.makedirs(OUTDIR)
    
SEED = 20

In [7]:
Nf = 20
nv_max = 3
random.seed(2)
true_sequence = np.array([np.full(nv_max, i) for i in range(Nf)]).flatten()
true_sequence = true_sequence.tolist()
random.shuffle(true_sequence)
print(true_sequence)

[16, 10, 13, 3, 5, 19, 2, 6, 12, 10, 14, 8, 13, 9, 2, 1, 6, 11, 7, 0, 7, 2, 8, 11, 19, 16, 15, 4, 16, 4, 3, 0, 14, 5, 15, 9, 11, 19, 10, 8, 13, 9, 17, 12, 0, 17, 4, 12, 5, 6, 14, 15, 3, 17, 7, 18, 1, 1, 18, 18]


In [10]:
# Register the environment so we can create it with gym.make()
gym.register(
    id=f"gymnasium_env/{env_name}",
    entry_point=SimpleTelEnv,
    max_episode_steps=300,  # Prevent infinite episodes. Here just set to 300 even though episode will terminate when stepping to last element of sequence
)

env = gym.make(f"gymnasium_env/{env_name}", Nf=Nf, target_sequence=true_sequence, nv_max=nv_max, off_by_lim=3)

from gymnasium.utils.env_checker import check_env

# This will catch many common issues
try:
    check_env(env.unwrapped)
    print("Environment passes all checks!")
except Exception as e:
    print(f"Environment has issues: {e}")

Environment passes all checks!


/home/hurra/.conda/envs/cosmoML/lib/python3.11/site-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment gymnasium_env/SimpleTel-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [19]:
env.action_space.n

np.int64(20)

In [11]:
from collections import defaultdict
import gymnasium as gym
import numpy as np


class QLearningAgent:
    def __init__(
        self,
        env: gym.Env,
        learning_rate: float,
        initial_epsilon: float,
        epsilon_decay: float,
        final_epsilon: float,
        discount_factor: float = 0.95,
    ):
        """Initialize a Q-Learning agent.

        Args:
            env: The training environment
            learning_rate: How quickly to update Q-values (0-1)
            initial_epsilon: Starting exploration rate (usually 1.0)
            epsilon_decay: How much to reduce epsilon each episode
            final_epsilon: Minimum exploration rate (usually 0.1)
            discount_factor: How much to value future rewards (0-1)
        """
        self.env = env

        # Q-table: maps (state, action) to expected reward
        # defaultdict automatically creates entries with zeros for new states
        self.q_values = defaultdict(lambda: np.zeros(env.action_space.n))

        self.lr = learning_rate
        self.discount_factor = discount_factor  # How much we care about future rewards

        # Exploration parameters
        self.epsilon = initial_epsilon
        self.epsilon_decay = epsilon_decay
        self.final_epsilon = final_epsilon

        # Track learning progress
        self.training_error = []

    def get_action(self, obs: tuple[int, int, bool]) -> int:
        """Choose an action using epsilon-greedy strategy.

        Returns:
            action: 0 (stand) or 1 (hit)
        """
        # With probability epsilon: explore (random action)
        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()

        # With probability (1-epsilon): exploit (best known action)
        else:
            return int(np.argmax(self.q_values[obs]))

    def update(
        self,
        obs: tuple[int, int, bool],
        action: int,
        reward: float,
        terminated: bool,
        next_obs: tuple[int, int, bool],
    ):
        """Update Q-value based on experience.

        This is the heart of Q-learning: learn from (state, action, reward, next_state)
        """
        # What's the best we could do from the next state?
        # (Zero if episode terminated - no future rewards possible)
        future_q_value = (not terminated) * np.max(self.q_values[next_obs])

        # What should the Q-value be? (Bellman equation)
        target = reward + self.discount_factor * future_q_value

        # How wrong was our current estimate?
        temporal_difference = target - self.q_values[obs][action]

        # Update our estimate in the direction of the error
        # Learning rate controls how big steps we take
        self.q_values[obs][action] = (
            self.q_values[obs][action] + self.lr * temporal_difference
        )

        # Track learning progress (useful for debugging)
        self.training_error.append(temporal_difference)

    def decay_epsilon(self):
        """Reduce exploration rate after each episode."""
        self.epsilon = max(self.final_epsilon, self.epsilon - self.epsilon_decay)

In [17]:
env.reset()

({'field_id': 16,
  'nvisits': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        dtype=int32),
  'index': 0},
 {'chosen_field_id': None, 'correct': None})

In [18]:
env.observation_space.sample()

{'field_id': np.int64(12),
 'index': np.int64(41),
 'nvisits': array([4, 2, 0, 4, 3, 2, 2, 2, 3, 1, 4, 2, 1, 4, 0, 2, 1, 4, 2, 3],
       dtype=int32)}

In [14]:
agent.update()

TypeError: QLearningAgent.update() missing 5 required positional arguments: 'obs', 'action', 'reward', 'terminated', and 'next_obs'

In [12]:
# Training hyperparameters
learning_rate = 0.01        # How fast to learn (higher = faster but less stable)
n_episodes = 100_000        # Number of hands to practice
start_epsilon = 1.0         # Start with 100% random actions
epsilon_decay = start_epsilon / (n_episodes / 2)  # Reduce exploration over time
final_epsilon = 0.1         # Always keep some exploration

agent = QLearningAgent(
    env=env,
    learning_rate=learning_rate,
    initial_epsilon=start_epsilon,
    epsilon_decay=epsilon_decay,
    final_epsilon=final_epsilon,
)


In [13]:
from tqdm import tqdm  # Progress bar

for episode in tqdm(range(n_episodes)):
    # Start a new hand
    obs, info = env.reset()
    done = False

    # Play one complete hand
    while not done:
        # Agent chooses action (initially random, gradually more intelligent)
        action = agent.get_action(obs)

        # Take action and observe result
        next_obs, reward, terminated, truncated, info = env.step(action)

        # Learn from this experience
        agent.update(obs, action, reward, terminated, next_obs)

        # Move to next state
        done = terminated or truncated
        obs = next_obs

    # Reduce exploration rate (agent becomes less random over time)
    agent.decay_epsilon()

  0%|          | 0/100000 [00:00<?, ?it/s]


TypeError: unhashable type: 'dict'

In [ ]:
# def generate_dataset(num_episodes=100_000):
#     grid_max = 10
#     ra_range = (-grid_max, grid_max)
#     dec_range = (-grid_max, grid_max)
#     n_points = grid_max
#     nvisits = 1
    
#     # generate random coords
#     ra_list = np.random.randint(ra_range[0], ra_range[1], size=(num_episodes, n_points))
#     dec_list = np.random.randint(dec_range[0], dec_range[1], size=(num_episodes, n_points))
#     coords = np.stack([ra_list, dec_list], axis=2) # shape (num_ep, nra_points, ndec_points)
#     coords_dict = {f'eps{i}': coord for i, coord in enumerate(coords)}
    
#     distance_matrices = np.empty(shape=(num_episodes, grid_max, grid_max))
#     # get distance matrices
#     for i in range(num_episodes):
#         distance_matrices[i] = get_distance_matrix(coords[i], coords[i])
#         np.fill_diagonal(distance_matrices[i], np.inf)
        
#     ordered_indices = np.argsort(distance_matrices, axis=2)
#     return coords, ordered_indices

#     start_ind = np.argmin(np.sum(coords**2, axis=2), axis=1)
    
#     target_indices = np.full(shape=(num_episodes, n_points), fill_value=-1, dtype=np.int32)
#     target_indices[:, 0] = start_ind

#     last_ind = start_ind
#     for i in range(num_episodes):
#         for k in range(len(coords[i]) - 1):
#             j = 0
#             current_ind = ordered_indices[i][last_ind[i]][j]
#             while current_ind in target_indices[i]:
#                 j += 1
#                 current_ind = ordered_indices[i][last_ind[i]][j]
#             target_indices[i, k] = current_ind
#             last_ind[i] = current_ind
#     return target_indices, coords
